<a href="https://colab.research.google.com/github/neonithinar/Datasets-for-my-Google-colab/blob/master/chapter_12_o_riley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom models and Loss Functions

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state = 42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state = 42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)



In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras


In [ ]:
def huber_fn(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

  

In [ ]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation= 'selu', kernel_initializer = 'lecun_normal', input_shape = input_shape), 
                                 keras.layers.Dense(10, activation = 'selu', kernel_initializer = 'lecun_normal'),
                                 keras.layers.Dense(1)])

model.compile(loss= huber_fn, optimizer = 'nadam', metrics= ['mae'])





In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 591
Trainable params: 591
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_scaled, y_train, epochs= 2, validation_data = (X_valid_scaled, y_valid))



Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4776 - mae: 0.8061 - val_loss: 0.2130 - val_mae: 0.4973
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2004 - mae: 0.4889 - val_loss: 0.1901 - val_mae: 0.4715


# Saving and loading models with custom components

In [ ]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [ ]:
model.compile(loss= create_huber(2.0), optimizer = 'nadam', metrics= ['mae'])


In [ ]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2160 - mae: 0.4797 - val_loss: 0.1986 - val_mae: 0.4509
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2073 - mae: 0.4698 - val_loss: 0.2134 - val_mae: 0.4565


In [ ]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [ ]:
model.fit(X_train_scaled, y_train, epochs= 2, validation_data = (X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.1988 - mean_absolute_error: 0.4601 - val_loss: 0.2019 - val_mean_absolute_error: 0.4436
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.1964 - mean_absolute_error: 0.4560 - val_loss: 0.2376 - val_mean_absolute_error: 0.4562


Creating a subclass of keras.losses.Loss to save the custom threshold


In [ ]:
class HuberLoss(keras.losses.Loss):
  def __init__(self, threshold, **kwargs):
    self.threshold = threshold
    super().__init__(**kwargs)
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.threshold
    squared_loss = tf.square(error) / 2
    linear_loss = self.threshold * tf.abs(error) - self.threshold **2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "threshold": self.threshold}
    

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation= 'selu', kernel_initializer='lecun_normal', input_shape=input_shape),
                                 keras.layers.Dense(1)

])

model.compile(loss= HuberLoss(2.0), optimizer='nadam', metrics= ['mae'])
model.fit(X_train_scaled, y_train, epochs= 2, validation_data= (X_valid_scaled, y_valid))


Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.9089 - mae: 1.0004 - val_loss: 0.4791 - val_mae: 0.6315
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2492 - mae: 0.5132 - val_loss: 0.3846 - val_mae: 0.5618
